In [28]:
import numpy as np
import pandas as pd
import sklearn 
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
import seaborn as sns
import matplotlib.pyplot as plt

In [29]:
df = pd.read_excel('CERAMICA_DATA.xlsx')
df.value_counts('point')

point
white           34
blue            32
massa           29
1_massa         17
3_white         13
5_blue          12
4_white          5
6_blue           5
massa_pink       4
2_massa          4
massa_white      4
5_white          3
3_blue           2
blue_dak         2
6_massa          2
3_massa          1
blue_dark        1
8_blue           1
blue2            1
massa_pink2      1
massa_white2     1
point            1
white2           1
Name: count, dtype: int64

In [30]:
df

,obrazec,point,Si,K,Ca,Ti,Mn,Fe,Co,Ni,Cu,Zn,Pb1,Pb2,As2,Rb,Sr,Sn,Cr
0,str_5_mid,1_massa,80,281,1394,153,98,4480,97,25,24,45,169,174,50,98,158,10,41
1,str_5_mid,3_white,172,268,202,86,90,215,93,99,161,127,18552,14319,179,101,120,274,84
2,str_5_mid,6_blue,172,344,290,115,103,418,296,205,190,174,21025,15871,198,101,107,302,86
3,str_5_big,1_massa,90,285,2137,182,127,4807,85,71,36,47,235,219,38,82,189,8,47
4,str_5_big,3_massa,88,281,2000,169,91,3827,96,151,15,56,513,445,33,69,180,9,64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,rot_9,blue,137,377,341,86,100,1105,1455,1359,156,115,16321,10139,504,120,140,292,78
172,rot_9,massa,52,122,1547,160,78,3850,59,22,38,30,310,285,47,80,215,6,37
173,rot_10,white,150,259,422,89,90,269,73,100,111,109,15691,11627,185,106,130,369,63
174,rot_10,blue,158,301,312,81,101,1417,1157,642,162,137,19475,11575,630,101,141,316,107


In [31]:
equalZero = df.copy()
equalZero[equalZero.columns[2:]] = pd.DataFrame([equalZero[col].apply(lambda x: x == '0') 
                                                 for col in equalZero.columns[2:]]).T


equalZeroRes = equalZero[equalZero.columns[2:]].sum(axis = 0)
if False not in (equalZero.iloc[0] == 0):
    print('Нет 0 в датафрейме - можно произвести экспоненциальную нормаировку')
equalZeroRes

Нет 0 в датафрейме - можно произвести экспоненциальную нормаировку


Si     0
K      0
Ca     0
Ti     0
Mn     0
Fe     0
Co     0
Ni     0
Cu     0
Zn     0
Pb1    0
Pb2    0
As2    0
Rb     0
Sr     0
Sn     0
Cr     0
dtype: int64

In [32]:
metals = df.columns[2:]
df[metals[0]].unique()

array([80, 172, 90, 88, 156, 178, 165, 87, 68, 143, 142, 145, 157, 82, 75,
       129, 123, 153, 162, 49, 137, 78, 128, 158, 45, 130, 67, 144, 125,
       63, 57, 146, 70, 135, 151, 136, 164, 73, 139, 65, 161, 86, 141,
       140, 148, 106, 108, 154, 74, 163, 155, 72, 152, 170, 81, 160, 150,
       147, 61, 171, 64, 95, 71, 133, 54, 124, 202, 93, 138, 62, 58, 97,
       168, 111, 134, 149, 'Si', 25, 183, 179, 85, 114, 56, 132, 174, 116,
       89, 43, 131, 52], dtype=object)

In [34]:
df_scaled = df.copy()

for metal in metals:
    df_scaled[metal] = df_scaled[metal].apply(lambda x: pd.to_numeric(x, errors= 'coerce'))
    df_scaled = df_scaled.dropna()
    minEl = df_scaled[metal].min()
    '''print(minEl, metal, end = '/n')'''
    df_scaled[metal] = df_scaled[metal].apply(lambda x: 1 - np.exp(1 - (x / minEl)))

df_scaled['obrazec'] = df_scaled['obrazec'].apply(lambda s: 1 if any(spot in s for spot in ['ams', 'rot', 'holl']) else 0)

print(df_scaled[metals[1]].dtype)
df_scaled[metals[1]].unique()

float64


array([0.98035288, 0.97531981, 0.99349437, 0.98168436, 0.88841654,
       0.95895074, 0.94370744, 0.93051655, 0.92004696, 0.80091732,
       0.9897343 , 0.99557751, 0.98200289, 0.97398606, 0.93291229,
       0.91117193, 0.99196992, 0.93172493, 0.98859478, 0.99549924,
       0.57668941, 0.99251412, 0.98879313, 0.88443183, 0.99641709,
       0.99393526, 0.94564837, 0.99043001, 0.98231587, 0.71724031,
       0.99825478, 0.46824847, 0.59839277, 0.9922468 , 0.97209494,
       0.9871045 , 0.96036609, 0.98351432, 0.98567307, 0.82999541,
       0.99008826, 0.82392445, 0.9970459 , 0.97892461, 0.7826651 ,
       0.99382792, 0.93745881, 0.99587722, 0.99434626, 0.99238163,
       0.98380102, 0.92414619, 0.98898802, 0.98776566, 0.90799984,
       0.9950866 , 0.99848331, 0.99302144, 0.99444459, 0.89035707,
       0.97574902, 0.81763669, 0.9095998 , 0.98687627, 0.91423466,
       0.85983182, 0.99414437, 0.90302803, 0.93635191, 0.99123371,
       0.87160343, 0.92546535, 0.8496421 , 0.73173755, 0.96555

In [35]:
df_scaled.info()
df_scaled
df_scaled['obrazec'].unique()
df_scaled

<class 'pandas.core.frame.DataFrame'>
Index: 175 entries, 0 to 175
Data columns (total 19 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   obrazec  175 non-null    int64  
 1   point    175 non-null    object 
 2   Si       175 non-null    float64
 3   K        175 non-null    float64
 4   Ca       175 non-null    float64
 5   Ti       175 non-null    float64
 6   Mn       175 non-null    float64
 7   Fe       175 non-null    float64
 8   Co       175 non-null    float64
 9   Ni       175 non-null    float64
 10  Cu       175 non-null    float64
 11  Zn       175 non-null    float64
 12  Pb1      175 non-null    float64
 13  Pb2      175 non-null    float64
 14  As2      175 non-null    float64
 15  Rb       175 non-null    float64
 16  Sr       175 non-null    float64
 17  Sn       175 non-null    float64
 18  Cr       175 non-null    float64
dtypes: float64(17), int64(1), object(1)
memory usage: 27.3+ KB


,obrazec,point,Si,K,Ca,Ti,Mn,Fe,Co,Ni,Cu,Zn,Pb1,Pb2,As2,Rb,Sr,Sn,Cr
0,0,1_massa,0.889197,0.980353,0.999996,0.998706,0.927895,1.000000,0.759492,0.830987,0.693279,0.914696,0.881119,0.857605,0.923574,0.927895,0.977356,0.903028,0.883963
1,0,3_white,0.997205,0.975320,0.610273,0.963117,0.903028,0.621855,0.734197,0.999955,0.999999,0.999845,1.000000,1.000000,0.999992,0.935477,0.928379,1.000000,0.995753
2,0,6_blue,0.997205,0.993494,0.832783,0.991348,0.940084,0.941273,0.998338,1.000000,1.000000,0.999996,1.000000,1.000000,0.999998,0.935477,0.893799,1.000000,0.996359
3,0,1_massa,0.925726,0.981684,1.000000,0.999696,0.975368,1.000000,0.675348,0.998981,0.896969,0.926860,0.964981,0.933587,0.819908,0.869585,0.991149,0.811124,0.926860
4,0,3_massa,0.919540,0.980353,1.000000,0.999419,0.906554,1.000000,0.753403,1.000000,0.304856,0.963399,0.999797,0.998559,0.742605,0.788928,0.988374,0.864665,0.980220
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,1,blue,0.988667,0.996354,0.897598,0.963117,0.933043,0.999892,1.000000,1.000000,0.999998,0.999609,1.000000,1.000000,1.000000,0.968078,0.960931,1.000000,0.993262
172,1,massa,0.660404,0.680293,0.999999,0.999088,0.848760,1.000000,0.378115,0.764123,0.914098,0.729557,0.991268,0.978301,0.905310,0.859558,0.995975,0.632121,0.842157
173,1,white,0.993262,0.971098,0.953004,0.968254,0.903028,0.769589,0.561765,0.999959,0.999887,0.999379,1.000000,1.000000,0.999995,0.946385,0.947102,1.000000,0.978638
174,1,blue,0.995107,0.986167,0.864665,0.952641,0.935477,0.999994,1.000000,1.000000,0.999999,0.999928,1.000000,1.000000,1.000000,0.935477,0.962097,1.000000,0.999276


In [36]:
X_train, X_test, y_train, y_test = train_test_split(df_scaled[metals], df_scaled['obrazec'], test_size= 0.3, random_state= 42)
y_train

69     0
36     0
85     0
22     1
145    1
      ..
71     0
106    1
14     1
92     0
102    1
Name: obrazec, Length: 122, dtype: int64

In [37]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

knn = KNeighborsClassifier()
scores = cross_val_score(knn, X_train, y_train, cv = 4)
print(np.mean(scores), np.std(scores))
knn.fit(X = X_train, y= y_train)
y_predict = knn.predict(X_test)

accuracy = accuracy_score(y_test, y_predict)
print(f"Accuracy: {accuracy:.4f}")


print(classification_report(y_test, y_predict))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_predict))
pd.concat([pd.Series(y_test), pd.Series(y_predict, index= y_test.index)], axis = 1)

0.7134408602150538 0.039577342683397285
Accuracy: 0.8491
              precision    recall  f1-score   support

           0       0.67      0.67      0.67        12
           1       0.90      0.90      0.90        41

    accuracy                           0.85        53
   macro avg       0.78      0.78      0.78        53
weighted avg       0.85      0.85      0.85        53

Confusion Matrix:
[[ 8  4]
 [ 4 37]]


,obrazec,0
157,1,1
146,1,1
101,0,0
127,1,1
142,1,1
140,1,1
42,1,1
15,1,1
126,1,1
65,0,0


In [12]:
type(y_test)

pandas.core.series.Series